Predicting Korfball Results with Statistical Modelling: Dixon-Coles and Time-Weighting
====
*This post describes two popular improvements to the standard Poisson model for football predictions applied to korfball, collectively known as the Dixon-Coles model*

[Rework of Predicting Football Results with Statistical Modelling: Dixon-Coles and Time-Weighting by David Sheehan](https://dashee87.github.io/football/python/predicting-football-results-with-statistical-modelling-dixon-coles-and-time-weighting/)

In an earlier post, I showed how to build a simple Poisson model to crudely predict the outcome of korfball matches. In the same way teams herald slight changes to their traditional plain coloured jerseys as ground breaking, I thought I'd show how the basic model could be tweaked and improved in order to achieve revolutionary status.

The changes are motivated by a combination of intuition and statistics. The [Dixon-Coles](http://web.math.ku.dk/~rolf/teaching/thesis/DixonColes.pdf) model (named after the paper's authors) corrects for the basic model's underestimation of draws and it also incorporated a time component so that recent matches are considered more important in calculating average goals rate. This isn't a particularly nodel idea for a blog post. There are numerous implementations of the Dixon-Coles model out there. Like any niche statistical modelling exercise, however, they are mostly available in R. I strongly recommend the excellent [opisthokonta blog](http://opisthokonta.net/?cat=48), especially if you're interested in more advanced models. 

## Data

We'll initially pull the match results for the EKL 2018/2019 season from [fixtureslive.com](https://w.fixtureslive.com/comp/57863/table/England-Korfball-National-League-EKL-Premier-Division-2018-19). This code is pretty much the same as last time.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import poisson, skellam
from scipy.optimize import minimize


DATA_DIR = '../data/'
ekl_1819 = pd.read_csv(DATA_DIR + 'all_data.csv', index_col=0)
ekl_1819 = ekl_1819[ekl_1819.Season == '2018/19'][['Home Team', 'Home Score', 'Away Team', 'Away Score']].copy()
ekl_1819.reset_index(inplace=True, drop=True)
ekl_1819.head()

,Home Team,Home Score,Away Team,Away Score
0,Bristol Thunder 1,20.0,Nomads 1,21.0
1,Kingfisher 1,23.0,KV 1,17.0
2,Trojans 1,21.0,Tornadoes 1,14.0
3,Cambridge Tigers 1,13.0,Birmingham City 1,15.0
4,Bec 1,16.0,Norwich Knights 1,22.0


## Basic Poisson Model
I won't spend too long on this model, as it was the subject of the [previous post](). Essentially, you treat the number of goals scored by each team as two independent Poisson distributions (henceforth called the Basic Poisson (BP) model). The shape of each distribution is determined by the average number of goals scored by that team. A little reminder on the mathematical definition of the Poisson distribution:

$$P(x) = \frac{e^{-\lambda}\lambda^{x}}{x!}, \lambda > 0$$

In our case, $\lambda$ represents the team's average or expected goal scoring rate. The Poisson distribution is a decent approximation of a team's scoring frequency. All of the model's discussed here agree on this point; the diagreement centres on how to calculate $\lambda_{\text{home}}$ and $\lambda_{\text{away}}$.

We can formulate the model in mathematical terms:

$$ P(X_{ij}=x, Y_{ij}=y) = \frac{e^{-\lambda}\lambda^{x}}{x!}\frac{e^{-\mu}\mu^y}{y!} \\
\text{where   } \lambda=\alpha_{i}\beta_{j}\gamma \text{     }  \mu=\alpha_{j}\beta{i} 
$$

In this equation, $i$ and $j$ refer to the home and away teams, respectively; $\alpha$ and $\beta$ denote each team's attack and defensive strength, respectively, while $\gamma$ represents the home advantage factor. So, we need to calculate $\alpha$ and $\beta$ for each team, as well as $\gamma$ (the home field advantage term - it's the same value for every team). As this was explained in the [previous post](), I'll just show the model output.


As an example, Bec (playing at home) would be expected to score 27.98 goals against Cambridge Tigers, while their opponents would get about 14.97 goals on average (I'm using the terms average and expected interchangeably). As each team is treated independently, we can construct a match score probability matrix.

First [Published by Maher in 1982](http://www.90minut.pl/misc/maher.pdf), the BP model still serves a good starting point from which you can add features that more closely reflect the reality. That brings us onto the Dixon-Coles (DC) model. 

## Dixon-Coles Model
In their [1997 paper](http://web.math.ku.dk/~rolf/teaching/thesis/DixonColes.pdf), Mark Dixon and Stuart Coles proposed two specific improvements to the BP model:

- Introduce an interaction term to correct underestimated frequency of low scoring matches
- Apply time decay component so that recent matches are weighted more strongly

The authors claim that low score results in football (0-0, 1-0 and 1-1) are inherently under-reported by the BP model. In the paper, they provide some analysis that supports their case - though I wouldn't call their approach particularly rigorous. The matrix below shows the average difference between actual and model predicted scorelines for the 2005/06 English Premier League season all the way up to the to 2017/19 season. Green cells imply the model underestimated those scorelines, while red cells suggest overestimation - the colour strength indicated the level of disagreement.

![image](https://dashee87.github.io/images/actual_model_diff.png)

There does seem to be an issue arround low scoring draws, though it is less apparent with 1-0 and 0-1 results. The Dixon-Coles (DC) model applies a correction to the BP model. It can be written in these mathematical terms:


$$
P(X_{i,j}=x, Y_{j,i}=y)=\tau_{\lambda,\mu}(x)\frac{e^{-\lambda}\lambda^{x}}{x!}\frac{e^{-\mu}\mu^{y}}{y!} \\
\text{where}\quad\lambda=\alpha_{i}\beta_{j}\gamma \quad \mu=\alpha_{j}\beta_{i} \\
\tau_{\lambda,\mu}(x,y) = 
\begin{cases}
        1 - \lambda\mu\rho  & \text{if }x=y=0 \\
        1 + \lambda\rho & \text{if } x=1, y=0 \\
        1 + \mu\rho & \text{if } x=0, y=1 \\
        1 - \rho & \text{if } x=y=1 \\
        1 & \text{otherwise}
\end{cases}
$$

The key difference over the BP model is the addition of the $\tau$ (tau) function. It is highly dependent on teh $\rho$ (rho) parameter, which controls the strength of the correction (note: setting $\rho=0$ equated to the standard BP model). We can easily convert $\tau_{\lambda,\mu}(x,y)$ to Python code.

In [6]:
def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1 - (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

Unfortunately, you can't just update your match score matrix with this funciton; you need to recalculate the various coefficients that go into the model. And unfortunately again, you can't just implement an off the shelf generalised linear model, as we did before. We have to construct the likelihood function and find the coefficients that maximise it - a technique known as [Maximum Likelihood Estimation](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation). With matches index $k=1,...,N$ and corresponding scores $(x_{k},y_{k})$, this is the likelihood function that we seek to maximise:

$$
L(\alpha_{i},\beta_{i}, \rho, \gamma, i = 1,\dots, n) = \prod_{k=1}^{N}\tau_{\lambda_{k},\mu_{k}}(x_{k},y_{k})\frac{e^{-\lambda}\lambda^{x_k}}{x_k!}\frac{e^{-\mu}\mu^{y_k}}{y_k!} \\
\text{where}\quad \lambda_k = \alpha_{i(k)}\beta_{j(k)}\gamma \quad \mu_k=\alpha_{j(k)}\beta_{i(k)}
$$

In this equation, $i(k)$ and $j(k)$ respectively denote the indices of the home and away teams in match $k$. For a few [different reasons](https://www.quora.com/Why-is-log-likelihood-so-widely-used-in-Machine-Learning) (numerical precision, practicality, etc.), we'll actually maximise the log-likelihood function. As the logarithm is a strictly increasing function (i.e. $\log(b) > \log(a) \ \forall \ b>a$), both likelihood functions are maximised at the same point. Also, recall that $\log(a\ b) = \log(a) +  \log(b)$. We can thus write the log-likelihood function in Python code.

In [7]:
def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x)
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) +
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

You may have noticed that `dc_log_like` included a transformation of $\lambda$ and $\mu$, where $\lambda = \exp(\alpha_i+\beta_j+\gamma)$ and $\mu = \exp(\alpha_j + \beta_i)$, so that we're essentially trying to calculate expected log goals. This is equivalent to the log [link function](https://en.wikipedia.org/wiki/Generalized_linear_model) in the previous BP glm the maximization algorithm should be easier as $\lambda, \mu > 0 \ \forall \ \alpha,\beta,\gamma$. Non-positive lambdas are not compatible with a Poisson distribution, so this would return warnings and/or errors during implementation. 

We're now ready to find the parameters that maximise the log likelihood function. Basically, you design a function that takes a set of model parameters as an argument. You set some initial values and potentially include some constraints and select the appropriate optimisation algorithm. I've opted for scipy's [minimise function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) (a possible alternative is [fmin](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html) - note: the functions seek to minimise the negative log-likelihood). It employs a process analogous to gradient descent, so that the algorithm iteratively converges to the optimal parameter set. The computation can be quite slow as it's forced to approximate the derivatives. If you're not as lazy as me, you could potentially speed it up by [manually constructing the partial derivatives](https://projekter.aau.dk/projekter/files/14466581/AssessingTheNumberOfGoalsInSoccerMatches.pdf#page=61).

In line with the original [Dixon Coles paper](http://web.math.ku.dk/~rolf/teaching/thesis/DixonColes.pdf) and the [opisthokonta blog](https://opisthokonta.net/?p=890), I've added the constraint that $\frac{1}{n}\sum_i\alpha_i=1$ (i.e. the average attack strength value is 1). This step isn't strictly necessary, but it means that it should return a unique solution (otherwise, the model would suffer from overparameterisation and each execution would return different coefficients).

Okay, we're ready to find the coefficients that maximise the log-likelihood function for the 2018/19 EKL season.

In [8]:
def solve_parameters(dataset, debug=False, init_vals=None, options={'disp': True, 'maxiter': 100},
                     constraints = [{'type': 'eq', 'fun': lambda x: sum(x[:20])-20}], **kwargs):
    teams = np.sort(dataset['Home Team'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['Away Team'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Something's not right")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1, (n_teams)), # attack strength
                                    np.random.uniform(0, -1, (n_teams)), # defence strength
                                    np.array([0, 1.0]) # rho (score correction), gamma (home advantage)
                                   ))
    def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
        lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x)
        return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) +
                np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))
    
    def estimate_parameters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams: (2*n_teams)]))
        rho, gamma = params[-2:]
        log_like = [dc_log_like(row['Home Score'], row['Away Score'], 
                                score_coefs[row['Home Team']], defend_coefs[row['Home Team']],
                                score_coefs[row['Away Team']], defend_coefs[row['Away Team']], rho, gamma)
                    for row in dataset.itertuples()]
        return -sum(log_like)
    
    opt_output = minimize(estimate_parameters, init_vals, options=options, constraints=constraints, **kwargs)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                       opt_output.x))